**TEAM 8**

* Rahul Namadev Chavan   CB.EN.P2DSC21021
* Vinayak M              CB.EN.P2DSC21029


First we are importing all the necessary libraries


*   unicodedata:- This module provides access to the Unicode Character Databas (UCD) which defines character properties for all Unicode characters.
*  re:- for regular expression
*  numpy:- for n dimensional arrays
*  io:- it allows us to manage the file-related input and output operations
*  time:- it returns the number of seconds passed since epoch

In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

# We are downloading the dataset that contains language translation pairs in the format
*  Example:- I gave way to their demands.	Cedí a sus demandas.

In [2]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

--2022-06-30 14:07:28--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2022-06-30 14:07:28 (254 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



# We are checking all the directories or list of files present in the downloaded zip

In [3]:
ls

sample_data/  spa-eng.zip


# we are unzipping the file which we had downloaded

In [4]:
!unzip spa-eng.zip

Archive:  spa-eng.zip
   creating: spa-eng/
  inflating: spa-eng/_about.txt      
  inflating: spa-eng/spa.txt         


# After unzipping we will check all files and sub directories present in it 

In [5]:
ls

sample_data/  spa-eng/  spa-eng.zip


# we are changing the directory to spa-eng since dataset is present in this

In [6]:
cd spa-eng

/content/spa-eng


In [7]:
ls

_about.txt  spa.txt


# We are printing the dataset

In [8]:
!cat spa.txt

Streaming output truncated to the last 5000 lines.
Tom and I have big plans.	Tom y yo tenemos grandes planes.
Tom and Mary are cousins.	Tom y Mary son primos.
Tom and Mary are dancing.	Tom y Mary están bailando.
Tom answered my question.	Tom respondió a mi pregunta.
Tom apologized on Monday.	Tom se disculpó el lunes.
Tom arrived at the hotel.	Tom llegó al hotel.
Tom arrived home at 2:30.	Tom llegó a casa a las dos y media.
Tom arrived this morning.	Tom llegó esta mañana.
Tom asked for a discount.	Tom pidió un descuento.
Tom asked for directions.	Tom pidió direcciones.
Tom asked for some money.	Tom pidió algo de dinero.
Tom attended the wedding.	Tom asistió a la boda.
Tom avenged Mary's death.	Tom vengó la muerte de María.
Tom avoided the question.	Tom evitó la pregunta.
Tom began to tickle Mary.	Tom comenzó a hacerle gracia a Mary.
Tom bet $300 on the game.	Tom apostó $300 dólares en el juego.
Tom bet $300 on the race.	Tom apostó $300 dólares en la carrera.
Tom bought Mary a camera.	To

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
cd ..

/content


We first lower case all the words present and we strip them so that we can add start and end token to thee sentence. So that model will get to know when the sentence will start and when  it will end for predicting.

In [10]:
# Converts the unicode file to ascii
def preprocess_sentence(w):
  w = w.lower().strip()
  
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

We have 2 variables as example to show in which the english and spanish sentence is stored and we are printing it.

In [11]:
en_sentence = "I go home right after work.	"
sp_sentence = "Voy a casa inmediatamente después del trabajo."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence))

<start> i go home right after work. <end>
<start> voy a casa inmediatamente después del trabajo. <end>


The text sentences are almost clean, they are simple plain text, so we only need to remove accents. Since it is huge file and we are reading each sentence separately line by line with utf-8 encoding and strip them and split them line by line and we are returning the word pair english and spanish for each line

In [12]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [13]:
path_to_file = 'spa-eng/spa.txt'
en, sp = create_dataset(path_to_file, 3000)
print(en[-1])
print(sp[-1])

<start> i'm very hot. <end>
<start> estoy muy cachondo. <end>


* lang stores the list of sentences in a language 
* lang_tokenizer allows to vectorize a text corpus, by turning each text into either a sequence of integers or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf. 
* Then we are trying to fit the internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency.
* the texts_to_sequence transforms each text in texts to a sequence of integers. So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.
* when you feed the sequences into a neural network to train a model, the sequences all need to be uniform in size. Currently the sequences have varied lengths, so the next step is to make them all be the same size, by padding them with zeros. Using tf.keras.preprocessing.sequence.pad_sequences to add zeros to the sequences to make them all be the same length. By default, the padding goes at the start of the sequences, but here we have specified to pad at the end.

In [14]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

Here we are creating the cleaned input and output pair dataset

In [15]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

* We are Limiting the size of the dataset to experiment faster. Since it will take long time to train the complete dataset we will limit the dataset to 3000 so that it can be faster.
* Calculate max_length of input and output tensor. Here, we'll set those to the longest sentence in the dataset


In [16]:
num_examples = 3000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

* Here we are creating the training and validation sets using 80-20 spit and printing the length of each input_tensor_train,target_tensor_train,input_tensor_val and target_tensor_val

In [17]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

2400 2400 600 600


*  we are mapping the words to some index values as a means for representing each of these values.

In [18]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [19]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[1])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[1])

Input Language; index to word mapping
1 ----> <start>
59 ----> estaba
708 ----> equivocado.
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
3 ----> i
32 ----> was
631 ----> wrong.
2 ----> <end>


We will then proceed to define a few parameters that will be useful for the training procedure and the overall preparation of the dataset like:-
* Batch_size
* steps per epoch
* embedding dimension
* number of units
* vocabulary input and target size

In [20]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1  
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 9]), TensorShape([64, 6]))

* We are creating Encode Class for the encoder architecture through model subclassing method.
*  __init__ function, the initialize states block, and the call blocks. In the init block, we will define and initialize all the required parameters.
* In the call block of the encoder, we define a function such that it takes an input sequence and the initial states of the encoder. The input of the input sequences is passed through the embedding layer, and finally, the output of the embedding layer is passed into the Encoder GRU. The call function returns all the outputs of the encoder as well as the last time steps of the hidden and cell states.
* The initialize_hidden_state function is used to represent the initialization of the hidden states and initial cell states accordingly with respect to the batch size assigned to it. 

In [22]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

By calling the Encoder class we are giving the simple_hidden input from initialize_hidden_state() function and after taking the input we are printing the Encoder output shape and hidden state shape.

In [23]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 9, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


* After completion of encoder architecture we are going for dot attention mechanism this is the general method (self.w1=tf.keras.layers.Dense(units).
* The Attention class is used to operate on the provided scoring function.
* The first function in the Attention Class is used to initialize the weights and prepare the call function for the dot attention procedure.
* the Attention mechanism takes mainly two inputs for the current step. These two variables include the hidden states of the decoder and all the outputs of the encoder.
* we will find the score or similarity between the hidden states of the decoder and the encoder outputs. We will then proceed to multiply the score function with the encoder outputs to get the context vector. This function will finally return the context vector and attention weights.

In [24]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
    #score = np.matmul(self.query_with_time_axis, self.W1(values))
    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

* Here we are calling BahdanauAttention and printing the result shape and weight shape 

In [25]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 9, 1)


1.  Pass the vocab_size and embedding_dim to the embedding layer and then get the output
2.  Using the enc_output and decoder hidden state, compute the context vector.
3.  Concatenate the context vector with the step-1 output.
4. Pass the Step-3 output to GRU and get the decoder output and states(hidden and cell state).
5. Pass the decoder output to dense layer(vocab size) and store the result into output.
6. Return the states from step-4, output from Step-5, attention weights from Step-2.



In [26]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

* Create the decoder and call the Decoder class by passing the following parameters and then we call sample_decoder_output for testing and we are printing shape of it.

In [27]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 1232)


* After the completion of the architectural build for our machine translation model, we need to define a few parameters that are required for the process of training the model. We will make use of the Adam optimizer and utilize the loss object as the Sparse Categorical Cross-entropy for computing the cross-entropy loss between the labels and predictions.


In [28]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

* Now we will use the teacher forcing method to train the model architecture accordingly. The gradient tape method is often used for training complex structures more easily.
* Teacher forcing works by using the actual or expected output from the training dataset at the current time step y(t) as input in the next time step X(t+1), rather than the output generated by the network.
1. Pass the input through the encoder which return encoder output and the encoder hidden state.
2. The encoder output, encoder hidden state and the decoder input (which is the start token) is passed to the decoder.
3. The decoder returns the predictions and the decoder hidden state.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use teacher forcing to decide the next input to the decoder.
6. Teacher forcing is the technique where the target word is passed as the next input to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [29]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

* We will implement our training procedure for a total of two epochs.
* We can observe that the loss has drastically reduced. The model has been trained well with a gradually reducing loss and an overall improvement in the accuracy of predictions.

In [30]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs

  print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.0587
Epoch 1 Loss 3.0717
Time taken for 1 epoch 69.42768263816833 sec

Epoch 2 Batch 0 Loss 2.5996
Epoch 2 Loss 2.3443
Time taken for 1 epoch 45.99227857589722 sec

